<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeagueScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from contextlib import closing
from operator import attrgetter
from pprint import pprint
from subprocess import DEVNULL
from subprocess import Popen
import time


# Team Dic

In [7]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmpDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126','名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmp2Dic = {'清水':'126', '福岡':'135'}

J12021TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '徳島':'toku', '福岡':'fuku', '鳥栖':'tosu', '大分':'oita'}

J12020TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '鳥栖':'tosu', '大分':'oita'}

J12019TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw',  'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '鳥栖':'tosu', '大分':'oita','松本': 'mats','磐田':'iwat'}


def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

get_keys_from_value(TeamDic,'276')

['札幌']

<select id="comparison1_team"><option value="0">チーム選択</option><option value="276">札幌</option><option value="31219">八戸</option><option value="30673">岩手</option><option value="238">仙台</option><option value="30310">秋田</option><option value="294">山形</option><option value="30674">福島</option><option value="120">鹿島</option><option value="176">水戸</option><option value="30305">栃木</option><option value="30103">群馬</option><option value="122">浦和</option><option value="199">大宮</option><option value="121">千葉</option><option value="132">柏</option><option value="270">FC東京</option><option value="123">東京Ｖ</option><option value="30532">町田</option><option value="86">川崎Ｆ</option><option value="124">横浜FM</option><option value="296">横浜FC</option><option value="30675">YS横浜</option><option value="130">湘南</option><option value="30676">相模原</option><option value="150">甲府</option><option value="30528">松本</option><option value="30535">長野</option><option value="193">新潟</option><option value="30308">富山</option><option value="30526">金沢</option><option value="126">清水</option><option value="131">磐田</option><option value="30677">藤枝</option><option value="31296">沼津</option><option value="127">名古屋</option><option value="30302">岐阜</option><option value="134">京都</option><option value="128">Ｇ大阪</option><option value="133">Ｃ大阪</option><option value="136">神戸</option><option value="30000">鳥取</option><option value="30314">岡山</option><option value="129">広島</option><option value="30851">山口</option><option value="30536">讃岐</option><option value="30116">徳島</option><option value="30148">愛媛</option><option value="30974">今治</option><option value="135">福岡</option><option value="30313">北九州</option><option value="269">鳥栖</option><option value="30531">長崎</option><option value="30303">熊本</option><option value="207">大分</option><option value="31508">宮崎</option><option value="31042">鹿児島</option><option value="30317">琉球</option></select>

# ScrapeData

In [8]:
def scrapeData(homeTeamID,awayTeamID,year):
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)

    #試合結果を取得
    html = requests.get(url)

    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    time.sleep(1) 

    return df,texts

# Scrape Season Data

In [9]:
def scrapeSeasonData(TeamNameDic,dfList,year):
    for team in TeamNameDic:
        url = 'https://www.football-lab.jp/' +  TeamNameDic[team] + '/match/' + '?year=' + year
        #表を取得
        df = pd.read_html(url)[0]
        #最後の列を削除
        if (year == '2021'):
            df = df.drop(38).drop(df.columns[[2,6,7,8,19,20]], axis=1)
        else:
            df = df.drop(34).drop(df.columns[[2,6,7,8,19,20]], axis=1)
        #homeTeam名追加
        df.insert(2,'Team',team)
        #列名を英語に
        df = df.rename(columns={'相手': 'awayTeam','Unnamed: 5':'H or A','チャンス構築率':'CBP','シュート':'shot','シュート成功率':'shot acc','支配率':'possession','攻撃CBP':'ACBP','パスCBP':'PassCBP','奪取P':'InterceptP','守備P':'DP'})
        #Home Awayのチーム並びかえ
        df['Team'].where(df['H or A']=='H', df['awayTeam'] ,inplace = True)
        df['awayTeam'].where(df['H or A']=='H', team ,inplace = True)
        #Home Awayのスコアに修正
        df.insert(4,'awayScore',0)
        df.insert(4,'homeScore',0)
        #Teamがhomeの時のscore入力
        df['homeScore'].where(df['H or A']=='A',df['スコア'].str[:1],inplace = True)
        df['awayScore'].where(df['H or A']=='A',df['スコア'].str[2:3],inplace = True)
        #Teamがawayの時のscore入力
        df['awayScore'].where(df['H or A']=='H',df['スコア'].str[:1],inplace = True)
        df['homeScore'].where(df['H or A']=='H',df['スコア'].str[2:3],inplace = True)
        df = df.drop(df.columns[[6]], axis=1)
        dfList.append(df)
        time.sleep(1) 
    return dfList

In [ ]:
tmpJ12020TeamDic = {'札幌':'sapp', '仙台':'send',}
dfList = []
dfList =  scrapeSeasonData(tmpJ12019TeamDic,dfList,'2020')
dfList[0]

In [ ]:
SeasonData = []
SeasonData =  scrapeSeasonData(J12019TeamDic,SeasonData,'2019')
SeasonData

In [ ]:
tmp = SeasonData[0].merge(SeasonData[1], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
tmp2 = SeasonData[0].merge(SeasonData[2], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
tmp3 = tmp.merge(tmp2, how="outer")
tmp3

In [12]:
seasonDataList = []
for i in range (len(SeasonData)):
    if(i!=0):
        tmp0 = SeasonData[i].merge(SeasonData[0], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=1):
        tmp1 = SeasonData[i].merge(SeasonData[1], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=2):
        tmp2 = SeasonData[i].merge(SeasonData[2], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=3):
        tmp3 = SeasonData[i].merge(SeasonData[3], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=4):
        tmp4 = SeasonData[i].merge(SeasonData[4], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=5):
        tmp5 = SeasonData[i].merge(SeasonData[5], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=6):
        tmp6 = SeasonData[i].merge(SeasonData[6], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=7):
        tmp7 = SeasonData[i].merge(SeasonData[7], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=8):
        tmp8 = SeasonData[i].merge(SeasonData[8], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=9):
        tmp9 = SeasonData[i].merge(SeasonData[9], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=10):
        tmp10 = SeasonData[i].merge(SeasonData[10], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=11):
        tmp11 = SeasonData[i].merge(SeasonData[11], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=12):
        tmp12 = SeasonData[i].merge(SeasonData[12], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=13):
        tmp13 = SeasonData[i].merge(SeasonData[13], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=14):
        tmp14 = SeasonData[i].merge(SeasonData[14], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=15):
        tmp15 = SeasonData[i].merge(SeasonData[15], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=16):
        tmp16 = SeasonData[i].merge(SeasonData[16], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=17):
        tmp17= SeasonData[i].merge(SeasonData[17], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(len(SeasonData) ==19):
        if(i!=18):
            tmp18 = SeasonData[i].merge(SeasonData[18], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
        if(i!=19):
            tmp19 = SeasonData[i].merge(SeasonData[19], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=1 and i!=2):
        tmp = tmp1.merge(tmp2, how="outer")
    if(i!=0 and i!=2):
        tmp = tmp0.merge(tmp2, how="outer")
    if(i!=0 and i!=1):
        tmp = tmp0.merge(tmp1, how="outer")
    if(i!=0 and i!=1 and i!=2):
        tmp = tmp.merge(tmp2, how="outer")
    if(i!=3):
        tmp = tmp.merge(tmp3, how="outer")
    if(i!=4):
        tmp = tmp.merge(tmp4, how="outer")
    if(i!=5):
        tmp = tmp.merge(tmp5, how="outer")
    if(i!=6):
        tmp = tmp.merge(tmp6, how="outer")
    if(i!=7):
        tmp = tmp.merge(tmp7, how="outer")
    if(i!=8):
        tmp = tmp.merge(tmp8, how="outer")
    if(i!=9):
        tmp = tmp.merge(tmp9, how="outer")
    if(i!=10):
        tmp = tmp.merge(tmp10, how="outer")
    if(i!=11):
        tmp = tmp.merge(tmp11, how="outer")
    if(i!=12):
        tmp = tmp.merge(tmp12, how="outer")
    if(i!=13):
        tmp = tmp.merge(tmp13, how="outer")
    if(i!=14):
        tmp = tmp.merge(tmp14, how="outer")
    if(i!=15):
        tmp = tmp.merge(tmp15, how="outer")
    if(i!=16):
        tmp = tmp.merge(tmp16, how="outer")
    if(i!=17):
        tmp = tmp.merge(tmp17, how="outer")
    if(len(SeasonData) ==19):
        if(i!=18):
            tmp = tmp.merge(tmp18, how="outer")
        if(i!=19):
            tmp = tmp.merge(tmp19, how="outer")
    tmp = tmp.dropna(how='any')
    if(i!=0):
        seasonData = pd.concat([seasonData,tmp])
    else:
        seasonData = tmp
#.merge(tmp6, how="outer").merge(tmp7, how="outer").merge(tmp8, how="outer").merge(tmp9, how="outer").merge(tmp10, how="outer").merge(tmp11, how="outer").merge(tmp12, how="outer").merge(tmp13, how="outer").merge(tmp14, how="outer").merge(tmp15, how="outer").merge(tmp16, how="outer").merge(tmp17, how="outer").merge(tmp18, how="outer").merge(tmp19, how="outer")
seasonData

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,...,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
16,17,6.30,仙台,札幌,2,1,A,57,54,12.6%,...,46,43,13.8%,15,13.3%,41.7%,11.52,8.23,86.84,22.69
25,26,9.14,札幌,仙台,1,3,H,53,51,7.1%,...,49,47,13.8%,16,18.8%,36.3%,8.71,6.50,97.05,20.80
34,4,3.17,札幌,鹿島,1,3,H,47,49,11.9%,...,51,53,13.6%,14,21.4%,46.1%,15.10,10.04,73.75,15.70
37,27,9.28,鹿島,札幌,1,1,A,43,49,11.7%,...,51,57,10.2%,13,7.7%,59.7%,18.13,15.48,98.95,13.98
38,2,3.2,浦和,札幌,0,2,A,48,51,18.9%,...,49,52,11.7%,16,0.0%,47.3%,13.98,10.22,94.96,10.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,29,10.19,磐田,鳥栖,2,2,H,46,48,13.4%,...,52,54,6.6%,8,25.0%,42.8%,14.43,9.85,73.16,16.98
64,3,3.9,磐田,大分,1,2,H,19,58,6.6%,...,42,81,11.6%,14,14.3%,61.6%,17.83,14.12,93.70,13.17
65,27,9.28,大分,磐田,1,2,A,31,50,7.5%,...,50,69,14.3%,12,8.3%,53.6%,22.49,16.16,63.51,10.15
66,1,2.23,磐田,松本,1,1,H,44,46,12.3%,...,54,56,9.9%,14,7.1%,40.7%,9.51,8.45,99.26,16.23


In [13]:
seasonData.to_csv("2019SeasonData.csv",encoding = 'utf-8-sig')

In [14]:
csv ='/content/2019SeasonData.csv'
data = pd.read_csv(csv)

In [15]:
#index列を削除
data = data.drop(data.columns[[0]], axis=1)
data

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,...,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
0,17,6.30,仙台,札幌,2,1,A,57,54,12.6%,...,46,43,13.8%,15,13.3%,41.7%,11.52,8.23,86.84,22.69
1,26,9.14,札幌,仙台,1,3,H,53,51,7.1%,...,49,47,13.8%,16,18.8%,36.3%,8.71,6.50,97.05,20.80
2,4,3.17,札幌,鹿島,1,3,H,47,49,11.9%,...,51,53,13.6%,14,21.4%,46.1%,15.10,10.04,73.75,15.70
3,27,9.28,鹿島,札幌,1,1,A,43,49,11.7%,...,51,57,10.2%,13,7.7%,59.7%,18.13,15.48,98.95,13.98
4,2,3.20,浦和,札幌,0,2,A,48,51,18.9%,...,49,52,11.7%,16,0.0%,47.3%,13.98,10.22,94.96,10.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,29,10.19,磐田,鳥栖,2,2,H,46,48,13.4%,...,52,54,6.6%,8,25.0%,42.8%,14.43,9.85,73.16,16.98
608,3,3.90,磐田,大分,1,2,H,19,58,6.6%,...,42,81,11.6%,14,14.3%,61.6%,17.83,14.12,93.70,13.17
609,27,9.28,大分,磐田,1,2,A,31,50,7.5%,...,50,69,14.3%,12,8.3%,53.6%,22.49,16.16,63.51,10.15
610,1,2.23,磐田,松本,1,1,H,44,46,12.3%,...,54,56,9.9%,14,7.1%,40.7%,9.51,8.45,99.26,16.23


In [16]:
# Hが上に来る様に並び替え
data = data.sort_values('H or A_x', ascending=False)

In [17]:
#重複削除
data = data.drop_duplicates(subset=['節','開催日','Team','awayTeam'])

In [18]:
#節を昇順に並び替え
data = data.sort_values('節')
data

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,...,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
610,1,2.23,磐田,松本,1,1,H,44,46,12.3%,...,54,56,9.9%,14,7.1%,40.7%,9.51,8.45,99.26,16.23
238,1,2.23,湘南,札幌,2,0,H,53,46,8.9%,...,54,47,8.0%,11,0.0%,59.9%,23.86,17.82,104.53,16.01
38,1,2.23,仙台,浦和,0,0,H,47,49,6.7%,...,51,53,8.2%,10,0.0%,61.7%,19.00,16.50,81.44,10.15
178,1,2.23,川崎Ｆ,FC東京,0,0,H,66,48,14.2%,...,52,34,5.3%,6,0.0%,38.0%,10.79,9.42,83.39,17.86
96,1,2.23,鹿島,大分,1,2,H,72,63,15.2%,...,37,28,9.4%,11,18.2%,49.4%,11.39,10.41,91.38,21.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,34,12.70,大分,Ｃ大阪,0,2,H,46,50,8.1%,...,50,54,11.9%,12,16.7%,49.9%,21.30,19.25,79.83,14.36
311,34,12.70,名古屋,鹿島,0,1,H,42,49,7.0%,...,51,58,8.1%,10,0.0%,42.5%,12.48,9.76,76.96,17.42
213,34,12.70,横浜FM,FC東京,3,0,H,43,42,5.8%,...,58,57,6.1%,8,0.0%,43.8%,15.30,11.48,140.12,12.87
445,34,12.70,広島,仙台,1,0,H,76,68,15.1%,...,32,24,4.5%,5,0.0%,34.6%,7.74,6.94,90.10,23.01


In [19]:
#書き出し
data.to_csv("seasonData_2019.csv",encoding = 'utf-8-sig')